In [1]:
import duckdb
import pandas as pd
import numpy as np

In [ ]:
value = "wolv'; INSERT INTO Personnel (Name) VALUES ('Taylor Swift'); SELECT * FROM Species WHERE Code = 'wolv"

In [2]:
conn = duckdb.connect('database.db')
cur = conn.cursor()

In [4]:
# define the species query to get species that have egg data
species_query = """
SELECT DISTINCT Code, Scientific_name FROM Species
JOIN Bird_nests ON Species.Code = Bird_nests.Species
JOIN Bird_eggs USING(Nest_ID)
WHERE Width IS NOT NULL AND Length IS NOT NULL
ORDER BY scientific_name;
"""
# get distinct species in a df to look at the results formatted
species_df = pd.read_sql(species_query, conn)
species_df

/var/folders/x9/sw65_mxs0zvb9zdbf_h1m2r40000gp/T/ipykernel_53862/1837660417.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  species_df = pd.read_sql(species_query, conn)


,Code,Scientific_name
0,rutu,Arenaria interpres
1,dunl,Calidris alpina
2,wrsa,Calidris fuscicollis
3,sepl,Charadrius semipalmatus
4,reph,Phalaropus fulicarius
5,amgp,Pluvialis dominica
6,bbpl,Pluvialis squatarola


In [7]:
# use species query and fetch results
for row in conn.execute(species_query).fetchall():
    species_code = row[0]
    scientific_name = row[1]

    # define egg query to get volume data for each species code
    egg_query = f"""
    SELECT Width * Width * Length AS Volume FROM Species
    JOIN Bird_nests ON Species.Code = Bird_nests.Species
    JOIN Bird_eggs USING(Nest_ID)
    WHERE Code = ?
    AND Width IS NOT NULL AND Length IS NOT NULL
    """

    # get results of egg query in a df
    df = pd.read_sql(egg_query, conn, params=(species_code,))

    # calc the cv and print results with scientific name
    cv = round(df.Volume.std() / df.Volume.mean() * 100, 2)
    print(f"{scientific_name} {cv}%")

Arenaria interpres 21.12%
Calidris alpina 5.46%
Calidris fuscicollis 16.77%
Charadrius semipalmatus 8.99%
Phalaropus fulicarius 4.65%
Pluvialis dominica 19.88%
Pluvialis squatarola 6.94%


/var/folders/x9/sw65_mxs0zvb9zdbf_h1m2r40000gp/T/ipykernel_53862/707055633.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(egg_query, conn, params=(species_code,))


In [30]:
# Close connection
conn.close()